# Homework 2

**Name:** -- Agustin Villarreal --

**e-mail:** -- agustin.villarreal0743@alumnos.udg.mx --

# MODULES

In [5]:
# Load modules
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output
import math
import scipy.stats as stats


In [6]:
# Define 2Vec class
class Vec2d(object):
    """
    2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
    """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)
    
    def __str__(self):
        return f"Vec2d({self.x}, {self.y})"
    
    def __repr__(self):
        return f"Vec2d({self.x}, {self.y})"
    
    def to_tuple(self):
        return (self.x, self.y)

In [7]:
# Module 1: Correlated Random Walk
def generate_crw_trajectory(num_steps, speed, start_pos_x, start_pos_y, cauchy_coef, seed=None):
    """
    Generate a Correlated Random Walk trajectory
    
    Parameters:
    -----------
    num_steps : int
        Number of steps in the trajectory
    speed : float
        Speed/step size for the walker
    start_pos_x, start_pos_y : float
        Starting position coordinates
    cauchy_coef : float
        Cauchy distribution coefficient (0 < c < 1)
    seed : int, optional
        Random seed for reproducibility
        
    Returns:
    --------
    trajectory : numpy.ndarray
        Array of shape (num_steps, 3) with x, y, time coordinates
    """
    if seed is not None:
        np.random.seed(seed)
        
    # Initialize trajectory array
    trajectory = np.zeros((num_steps, 3))
    
    # Set starting position
    current_pos = Vec2d(start_pos_x, start_pos_y)
    
    # Initial direction (random)
    direction = Vec2d(1, 0)
    
    # Store initial position
    trajectory[0, 0] = current_pos.x
    trajectory[0, 1] = current_pos.y
    trajectory[0, 2] = 0  # time
    
    # Generate trajectory
    for i in range(1, num_steps):
        # Sample turning angle from Cauchy distribution
        angle = np.random.standard_cauchy() * cauchy_coef
        
        # Update direction with rotation
        direction = direction.rotated(angle)
        
        # Normalize direction and apply speed
        length = direction.get_length()
        if length > 0:
            direction = Vec2d(direction.x / length * speed, direction.y / length * speed)
        
        # Update position
        current_pos = current_pos + direction
        
        # Store position
        trajectory[i, 0] = current_pos.x
        trajectory[i, 1] = current_pos.y
        trajectory[i, 2] = i  # time
    
    return trajectory

def plot_crw_trajectory_2d(trajectory, title="Correlated Random Walk Trajectory"):
    """
    Plot a 2D projection of the CRW trajectory using Plotly
    
    Parameters:
    -----------
    trajectory : numpy.ndarray
        Array with x, y, time coordinates
    title : str
        Plot title
    
    Returns:
    --------
    fig : plotly.graph_objects.Figure
        Plotly figure object
    """
    fig = go.Figure()
    
    # Add trajectory line
    fig.add_trace(go.Scatter(
        x=trajectory[:, 0],
        y=trajectory[:, 1],
        mode='lines',
        name='CRW',
        line=dict(color='red', width=2)
    ))
    
    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title="x_pos (mm)",
        yaxis_title="y_pos (mm)",
        template="plotly_white"
    )
    
    return fig

def plot_crw_trajectory_3d(trajectory, title="Correlated Random Walk Trajectory"):
    """
    Plot a 3D projection of the CRW trajectory using Plotly
    
    Parameters:
    -----------
    trajectory : numpy.ndarray
        Array with x, y, time coordinates
    title : str
        Plot title
    
    Returns:
    --------
    fig : plotly.graph_objects.Figure
        Plotly figure object
    """
    fig = go.Figure()
    
    # Add trajectory line
    fig.add_trace(go.Scatter3d(
        x=trajectory[:, 0],
        y=trajectory[:, 1],
        z=trajectory[:, 2],
        mode='lines',
        name='CRW',
        line=dict(color='red', width=4)
    ))
    
    # Update layout
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title="x_pos",
            yaxis_title="y_pos",
            zaxis_title="time",
        ),
        template="plotly_white"
    )
    
    return fig


In [8]:
# Module 2: Lévy Distribution
def levy_pdf(x, alpha, beta=0, mu=0, c=1):
    """
    Lévy stable distribution PDF approximation
    
    Parameters:
    -----------
    x : array_like
        Points at which to evaluate the PDF
    alpha : float
        Stability parameter (0 < alpha <= 2)
    beta : float, optional
        Skewness parameter (-1 <= beta <= 1)
    mu : float, optional
        Location parameter
    c : float, optional
        Scale parameter
        
    Returns:
    --------
    pdf : array_like
        PDF values at points x
    """
    # Using scipy's implementation of the stable distribution
    return stats.levy_stable.pdf(x, alpha, beta, loc=mu, scale=c)

def plot_levy_distributions(x_range, alphas, mu=3.0, beta=0.0):
    """
    Plot multiple Lévy distributions with different alpha values
    
    Parameters:
    -----------
    x_range : array_like
        Range of x values to plot
    alphas : list
        List of alpha values to plot
    mu : float, optional
        Location parameter
    beta : float, optional
        Skewness parameter
        
    Returns:
    --------
    fig : plotly.graph_objects.Figure
        Plotly figure object
    """
    fig = go.Figure()
    
    # Add a trace for each alpha value
    for alpha in alphas:
        pdf_values = levy_pdf(x_range, alpha, beta, mu)
        
        fig.add_trace(go.Scatter(
            x=x_range,
            y=pdf_values,
            mode='lines',
            name=f'Lévy {alpha}',
            line=dict(width=2)
        ))
    
    # Update layout
    fig.update_layout(
        title="Lévy Stable Distributions",
        xaxis_title="x",
        yaxis_title="Probability Density",
        template="plotly_white",
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="right",
            x=0.99
        )
    )
    
    return fig

In [ ]:
# Plot the results